# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [1]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
#!unzip data.zip
#!ls 

## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [2]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

train = train.reshape(1229932, 11, 39)
test = test.reshape(451552, 11, 39)

#feats = [0,2,3,4,5,6,7,8,10]
#train = train[:,feats,:]
#test = test[:,feats,:]

train = train.reshape(1229932, -1)
test = test.reshape(451552, -1)

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


In [3]:
#from scipy import spatial.distance.cosine
# 0,1: 0.8406975890047381
# 1,2: 0.8408058577925639
# 0,2: 0.5890390591353316
# 看來0,1 有點重疊->刪掉1
# 2,3: 0.8408435099576054
# 3,4: 0.8408087361471327
# 2,4: 0.5891934447436348
# 看來2,3 有點重疊->刪掉3
# 4,5: 0.8407781385646762
# 5,6: 0.8408703774927714
# 4,6: 0.5893461401252141
# 看來4,5 有點重疊
# 6,7: 0.8408710386399266
# 7,8: 0.8408018768428297
# 6,8: 0.5896318464957054
# 看來6,7 有點重疊->刪掉7
# 8,9: 0.8407466205531563
# 9,10: 0.840738934407591
# 8,10: 0.5898820477117614
# 看來8,9 有點重疊->刪掉9
# 用0,2,4,5,6,8,10共7個frame

## Create Dataset

In [4]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)

Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [5]:
VAL_RATIO = 0

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (1229932, 429)
Size of validation set: (0, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [6]:
BATCH_SIZE = 4096

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [7]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

40

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [8]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.net = nn.Sequential(
            nn.Linear(429, 2048), # 1
            nn.LeakyReLU(),
            #nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048, 2048), # 2
            nn.LeakyReLU(),
            #nn.ReLU(),
            nn.BatchNorm1d(2048),
            nn.Dropout(0.5),
            nn.Linear(2048,1024), # 3
            nn.LeakyReLU(),
            #nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, 512), # 4
            #nn.ReLU(),
            nn.LeakyReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.5),
            nn.Linear(512, 128), # 5
            #nn.ReLU(),
            nn.LeakyReLU(),
            nn.BatchNorm1d(128),
            nn.Dropout(0.5),
            nn.Linear(128, 39)
        )

    def forward(self, x):
        x = self.net(x)
        return x

## Training

In [9]:
#check device
def get_device():
    return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [10]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [11]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 300              # number of training epoch
learning_rate = 1e-4       # learning rate
l2 = 1e-3
# the path where checkpoint saved
model_path = './model2dnn.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=l2)

DEVICE: cuda


In [12]:
# start training
best_acc = 0.0
epoch = 0
    
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_acc': train_acc/len(train_set),
                    'train_loss': train_loss/len(train_loader),
                    'val_acc': best_acc/len(val_set),
                    'val_loss': val_loss/len(val_loader),
                }, model_path)
                #torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))

    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))
        epoch += 1

# if not validating, save the last epoch
if len(val_set) == 0:
    #torch.save(model.state_dict(), model_path)
    torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_acc': train_acc/len(train_set),
                    'train_loss': train_loss/len(train_loader),
                }, model_path)
    print('saving model at last epoch')
    
print('Finished training after {} epochs'.format(epoch))

[001/300] Train Acc: 0.387516 Loss: 2.318095
[002/300] Train Acc: 0.522996 Loss: 1.683480
[003/300] Train Acc: 0.569180 Loss: 1.493301
[004/300] Train Acc: 0.596703 Loss: 1.384131
[005/300] Train Acc: 0.614618 Loss: 1.313169
[006/300] Train Acc: 0.628923 Loss: 1.259490
[007/300] Train Acc: 0.640187 Loss: 1.217058
[008/300] Train Acc: 0.649010 Loss: 1.182186
[009/300] Train Acc: 0.657712 Loss: 1.151694
[010/300] Train Acc: 0.663449 Loss: 1.128226
[011/300] Train Acc: 0.669684 Loss: 1.105447
[012/300] Train Acc: 0.674982 Loss: 1.085353
[013/300] Train Acc: 0.679652 Loss: 1.067213
[014/300] Train Acc: 0.683893 Loss: 1.051700
[015/300] Train Acc: 0.688370 Loss: 1.034933
[016/300] Train Acc: 0.692159 Loss: 1.021500
[017/300] Train Acc: 0.696225 Loss: 1.008175
[018/300] Train Acc: 0.700069 Loss: 0.994167
[019/300] Train Acc: 0.703098 Loss: 0.981687
[020/300] Train Acc: 0.705458 Loss: 0.971647
[021/300] Train Acc: 0.708955 Loss: 0.960487
[022/300] Train Acc: 0.711342 Loss: 0.950439
[023/300] 

In [13]:
def retrain(model_path, num_epoch):
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    n_epoch = checkpoint['epoch']
    train_acc = checkpoint['train_acc']
    train_loss = checkpoint['train_loss']
    print('Retrain %d epochs'% num_epoch)
    if len(val_set) > 0:
        val_acc = checkpoint['val_acc']
        val_loss = checkpoint['val_loss']
        print('Resume from epoch {:03d} Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                n_epoch , train_acc, train_loss, val_acc, val_loss
            ))
    else:
        print('Resume from epoch {:03d} Train Acc: {:3.6f} Loss: {:3.6f}'.format(
                n_epoch, train_acc, train_loss
            ))

    
    # start training
    best_acc = 0.0
    epoch = 0
    num_epoch+=n_epoch
    for epoch in range(n_epoch, num_epoch):
        train_acc = 0.0
        train_loss = 0.0
        val_acc = 0.0
        val_loss = 0.0

        # training
        model.train() # set the model to training mode
        for i, data in enumerate(train_loader):
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad() 
            outputs = model(inputs) 
            batch_loss = criterion(outputs, labels)
            _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
            batch_loss.backward() 
            optimizer.step() 

            train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
            train_loss += batch_loss.item()

        # validation
        if len(val_set) > 0:
            model.eval() # set the model to evaluation mode
            with torch.no_grad():
                for i, data in enumerate(val_loader):
                    inputs, labels = data
                    inputs, labels = inputs.to(device), labels.to(device)
                    outputs = model(inputs)
                    batch_loss = criterion(outputs, labels) 
                    _, val_pred = torch.max(outputs, 1) 

                    val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                    val_loss += batch_loss.item()

                print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                    epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
                ))

                # if the model improves, save a checkpoint at this epoch
                if val_acc > best_acc:
                    best_acc = val_acc
                    torch.save({
                        'epoch': epoch,
                        'model_state_dict': model.state_dict(),
                        'optimizer_state_dict': optimizer.state_dict(),
                        'train_acc': train_acc/len(train_set),
                        'train_loss': train_loss/len(train_loader),
                        'val_acc': best_acc/len(val_set),
                        'val_loss': val_loss/len(val_loader),
                    }, model_path)
                    #torch.save(model.state_dict(), model_path)
                    print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))

        else:
            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
            ))
            epoch += 1

    # if not validating, save the last epoch
    if len(val_set) == 0:
        #torch.save(model.state_dict(), model_path)
        torch.save({
                    'epoch': epoch,
                    'model_state_dict': model.state_dict(),
                    'optimizer_state_dict': optimizer.state_dict(),
                    'train_acc': train_acc/len(train_set),
                    'train_loss': train_loss/len(train_loader),
                }, model_path)
        print('saving model at last epoch')

    print('Finished training after {} epochs'.format(epoch))

In [14]:
#retrain(model_path, num_epoch=40)

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [15]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
#model.load_state_dict(torch.load(model_path))
model.load_state_dict(torch.load(model_path)['model_state_dict'])

<All keys matched successfully>

Make prediction.

In [16]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [17]:
count = 0
for i in range(1, len(predict)-1):
    previous_ = predict[i-1]
    next_ = predict[i+1]
    current_ = predict[i]
    if (previous_ != current_) and (next_ != current_) and (previous_ == next_):
        print('idx',i,'correct', current_, 'to', previous_)
        predict[i] = previous_
        count +=1

print('total number of correction %d, correction percent %.2f'% (count, count/len(predict)))

idx 563 correct 11 to 3
idx 597 correct 3 to 11
idx 604 correct 7 to 3
idx 669 correct 0 to 1
idx 753 correct 2 to 1
idx 968 correct 11 to 3
idx 1046 correct 19 to 1
idx 1187 correct 38 to 33
idx 1266 correct 4 to 12
idx 1287 correct 7 to 4
idx 1378 correct 38 to 33
idx 1446 correct 13 to 12
idx 1998 correct 14 to 17
idx 2105 correct 21 to 36
idx 2367 correct 4 to 12
idx 2454 correct 4 to 12
idx 2540 correct 20 to 19
idx 2794 correct 0 to 5
idx 2825 correct 25 to 38
idx 2995 correct 12 to 5
idx 3049 correct 38 to 19
idx 3273 correct 13 to 7
idx 3305 correct 1 to 0
idx 3337 correct 13 to 12
idx 3472 correct 2 to 1
idx 3636 correct 3 to 9
idx 3706 correct 37 to 0
idx 3740 correct 19 to 20
idx 3754 correct 8 to 3
idx 4001 correct 8 to 3
idx 4080 correct 33 to 28
idx 4127 correct 38 to 37
idx 4131 correct 33 to 37
idx 4349 correct 8 to 0
idx 4375 correct 37 to 38
idx 4710 correct 0 to 8
idx 4810 correct 32 to 38
idx 5245 correct 17 to 1
idx 5355 correct 14 to 17
idx 5769 correct 18 to 19
i

In [18]:
with open('prediction_dnn2.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))

# Reference
The code is modified from TA's sample code